# Simulator

In [1]:
import pandas as pd

import payment_simulator as ps
from payment_simulator.anomaly import AnomalyGenerator
from payment_simulator.networks import SimplePaymentNetwork
from payment_simulator.utils import random_payment_timing, random_payment_value

In [2]:
TOTAL_BANKS = 20
INITIAL_BANKS = 3
INCREMENT_BANKS = 5
ALPHA = 0.00001
AVG_PAYMENTS = 35
ALLOW_SELF_LOOP = False

In [3]:
sim_periods = list(range(15))

sim_params = {
    "open_time": "06:30:00",
    "close_time": "18:30:00",
    "value_fn": random_payment_value,
    "timing_fn": random_payment_timing,
}

payment_network = SimplePaymentNetwork(
    total_banks=TOTAL_BANKS,
    avg_payments=AVG_PAYMENTS,
    alpha=ALPHA,
    allow_self_loop=ALLOW_SELF_LOOP
)

anomaly_generator = AnomalyGenerator(
    anomaly_start=5,
    anomaly_end=13,
    prob_start=0.8,
    prob_end=1,
    lambda_start=5.5,
    lambda_end=12.5,
    rate=0.5,
)

In [4]:
normal_transactions = ps.TransactionSim(sim_id=1, network=payment_network, **sim_params)
normal_transactions.run(sim_periods)

payments1 = normal_transactions.get_payments_df()

pd.concat([payments1.head(3), payments1.tail(3)])

Period      Time  Sender  Receiver  Count     Value
0           0  17:10:03       0         5      1  0.480899
1           0  13:25:06       0         5      1  2.948349
2           0  08:01:47       0         5      1  2.967845
10497      14  10:04:11      18         4      1  0.216577
10498      14  06:34:17      19        12      1  1.170413
10499      14  08:54:20      19        17      1  1.546033

In [5]:
anomaly_transactions = ps.TransactionSim(sim_id=2, network=payment_network, anomaly_gen=anomaly_generator, **sim_params)
anomaly_transactions.run(sim_periods, anomalous_bank = [1,3,5])

payments2 = anomaly_transactions.get_payments_df()

pd.concat([payments2.head(3), payments2.tail(3)])

Period      Time  Sender  Receiver  Count     Value
0           0  07:49:31       0         8      1  0.119766
1           0  13:03:15       0         8      1  0.059179
2           0  17:24:20       0         8      1  1.016385
10497      14  13:54:47      18         4      1  4.427932
10498      14  09:37:07      18         9      1  0.249910
10499      14  13:20:59      19         4      1  0.455802

In [6]:
print(f"Total Value of Normal RTGS  : {payments1['Value'].sum():.3f} from {payments1.shape[0]} transactions")
print(f"Total Value of Anomaly RTGS : {payments2['Value'].sum():.3f} from {payments1.shape[0]} transactions")

Total Value of Normal RTGS  : 17317.018 from 10500 transactions
Total Value of Anomaly RTGS : 32849.544 from 10500 transactions
